In [1]:
import adala

# Create agent


In [2]:
agent = adala.SingleShotAgent()
agent.greet()


Hi, I am a single shot agent.
I can use a single skill to process a dataset.
For example, you can instruct me to label your dataset.

I have no skills yet. To begin, you need to teach me a skill.
For example, try:

agent.skill = adala.LabelingSkill(labels=['positive', 'negative'])


# Add skill

In [3]:
agent.skill = adala.LabelingSkill(labels=['Subjective', 'Objective'])

# Run agent

In [4]:
result = agent.run()
print(result.experience.model_dump())

{'dataset': {}, 'predictions': Empty DataFrame
Columns: []
Index: [], 'evaluations': None, 'errors': None, 'accuracy': None, 'initial_instructions': None, 'updated_instructions': None}


As you can notice, agent's experience is empty, since no dataset specified. Let's connect agent to a dataset produced by Label Studio:

In [7]:
agent.dataset = adala.LabelStudioDataset(
    label_studio_url='http://localhost:8080',
    label_studio_api_key='681842051079710a6b0ebce5ec56f746e3400c98',
    label_studio_project_id=218
)
# agent.dataset.info()

Now agent can evoke predictions from the real dataset:

In [8]:
result_2 = agent.run()
print(result_2.experience.predictions)

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 71.64it/s]

       predictions                                              score
407783  Subjective  {'Subjective': -0.28115004, 'Objective': -1.40...
407784  Subjective  {'Subjective': -0.6623852999999998, 'Objective...
407785  Subjective  {'Subjective': -0.08893942999999996, 'Objectiv...
407786  Subjective  {'Subjective': -0.08893942999999996, 'Objectiv...
407787  Subjective  {'Subjective': -0.5759392999999999, 'Objective...
...            ...                                                ...
407843  Subjective  {'Subjective': -0.006715368000000036, 'Objecti...
407844  Subjective  {'Subjective': -0.058320418000000006, 'Objecti...
407845  Subjective  {'Subjective': -0.12692796000000006, 'Objectiv...
407846   Objective  {'Subjective': -0.75759894, 'Objective': -0.63...
407847  Subjective  {'Subjective': -0.09441284999999998, 'Objectiv...

[65 rows x 2 columns]


# Learning from experience
If ground truth information is presented in the dataset, agent can learn and improve.

In [9]:
learn_result = agent.learn()

100%|███████████████████████████████████████████████| 5/5 [00:00<00:00, 40.25it/s]


Evaluating, analyzing and improving...
Done!


For example, now it can use the updated instructions finetuned on the ground truth data:

In [10]:
print(f'''
Initial instructrions:
{learn_result.experience.initial_instructions}

Updated instructions:
{learn_result.experience.updated_instructions}
''')

print(f'Current accuracy: {learn_result.experience.accuracy}')


Initial instructrions:
Label the input text with the following labels: {{labels}}

Updated instructions:
Please categorize the input text based on its objectivity or subjectivity according to the following criteria: 
- Label as "Subjective" if the text represents personal opinions, feelings, or thoughts.
- Label as "Objective" if the text provides factual information, detailed description, or mentions specific features without expressing personal opinions, feelings, or sentiments.

Examples:

Input: {"index":76,"text":"Because both ears are occupied, background is not distracting at all.","label":"Positive","score":{"Subjective":-0.6623853,"Objective":-0.7248853},"predictions_match":false}
Output: Objective

Input: {"index":69,"text":"Battery is holding up well.","label":"Positive","score":{"Subjective":-0.6623853,"Objective":-0.7248853},"predictions_match":false}
Output: Objective

Input: {"index":48,"text":"Any ring tone..Overall good phone to buy..","label":"Positive","score":{"Sub